<a href="https://colab.research.google.com/github/Run-aivle/study/blob/main/March/03.29/SEN/Sequential(Bidirectional%2C_Conv1D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  데이터 불러오기

In [ ]:
pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import FinanceDataReader as fdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras as tf
from tensorflow import keras
from sklearn.metrics import mean_squared_error
from tensorflow.keras.utils import plot_model

In [ ]:
# 삼성전자 종목
data = fdr.DataReader('005930', '2015-01-08', '2023-03-24')
data.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2015-01-08,26780,26780,26199,26280,289552,0.005356
2015-01-09,26280,26440,26280,26280,186248,0.000000
2015-01-12,26260,26379,26020,26320,166508,0.001522
2015-01-13,26280,26800,26000,26780,245868,0.017477
2015-01-14,26780,27100,26700,26900,286645,0.004481


In [ ]:
data.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2023-03-20,61100,61200,60200,60200,9618009,-0.017945
2023-03-21,60500,60700,60100,60300,8318514,0.001661
2023-03-22,61000,61200,60500,61100,8978591,0.013267
2023-03-23,60600,62300,60600,62300,15381057,0.019640
2023-03-24,62700,63300,62300,63000,18278602,0.011236


# 데이터 전처리

In [ ]:
data.reset_index(inplace = True)
data.head()

,Date,Open,High,Low,Close,Volume,Change
0,2015-01-08,26780,26780,26199,26280,289552,0.005356
1,2015-01-09,26280,26440,26280,26280,186248,0.000000
2,2015-01-12,26260,26379,26020,26320,166508,0.001522
3,2015-01-13,26280,26800,26000,26780,245868,0.017477
4,2015-01-14,26780,27100,26700,26900,286645,0.004481


In [ ]:
data.drop('Date',axis = 1, inplace = True)
data.head()

,Open,High,Low,Close,Volume,Change
0,26780,26780,26199,26280,289552,0.005356
1,26280,26440,26280,26280,186248,0.000000
2,26260,26379,26020,26320,166508,0.001522
3,26280,26800,26000,26780,245868,0.017477
4,26780,27100,26700,26900,286645,0.004481


In [ ]:
data.tail()

,Open,High,Low,Close,Volume,Change
2017,61100,61200,60200,60200,9618009,-0.017945
2018,60500,60700,60100,60300,8318514,0.001661
2019,61000,61200,60500,61100,8978591,0.013267
2020,60600,62300,60600,62300,15381057,0.019640
2021,62700,63300,62300,63000,18278602,0.011236


In [ ]:
data = data.values
data.shape

(2022, 6)

In [ ]:
timestep = 10

x = np.array([data[i : i+timestep] for i in range(len(data)-timestep)])
y = np.array([data[i+timestep, -3] for i in range(len(data) - timestep)])

x_train, x_val, x_test = x[:-15], x[-15:-5], x[-5:]
y_train, y_val, y_test = y[:-15], y[-15:-5], y[-5:]

print(x.shape, y.shape)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2012, 10, 6) (2012,)
(1997, 10, 6) (1997,)
(10, 10, 6) (1997,)
(5, 10, 6) (5,)


# 모델 학습

In [ ]:
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GRU, LSTM, Conv1D, MaxPool1D, Bidirectional
from sklearn.metrics import mean_squared_error as MSE

In [ ]:
# 세션 클리어
clear_session()

# 모델 layer 엮기
il = Input(shape = (10, 6))
hl1 = Conv1D(filters = 32,
             kernel_size = 3,
             strides = 1,
             padding = 'valid',
             activation = 'relu')(il)
hl2 = MaxPool1D(pool_size = 2,
                strides = 2)(hl1)
hl3 = Bidirectional(layer = LSTM(32, return_sequences=True))(hl2)
hl4 = Bidirectional(layer = GRU(16, return_sequences=True))(hl3)

# 정방향은 GRU, 역방향은 LSTM
forward_GRU = GRU(24, return_sequences= True)
backward_LSTM = LSTM(24 ,return_sequences=True, go_backwards=True)
hl5 = Bidirectional(layer = forward_GRU, backward_layer=backward_LSTM)(hl4)
hl6 = Flatten()(hl5)

# 종가이므로 0 값이 나오지 않도록 방지
ol = Dense(1, activation = 'relu')(hl6)

# model 처음과 끝 지정
model = Model(il, ol)

# model compile
model.compile(loss = 'mae',
              optimizer = 'adam')

# 양념
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 6)]           0         
                                                                 
 conv1d (Conv1D)             (None, 8, 32)             608       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4, 32)            0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 4, 64)            16640     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 4, 32)            7872      
 nal)                                                            
                                                             

In [ ]:
model.fit(x_train, y_train,
          epochs = 30, validation_data = [x_val, y_val])

Epoch 1/30
63/63 [==============================] - 17s 55ms/step - loss: 49784.4961 - val_loss: 60194.1758
Epoch 2/30
63/63 [==============================] - 1s 16ms/step - loss: 49766.0078 - val_loss: 60176.7188
Epoch 3/30
63/63 [==============================] - 1s 17ms/step - loss: 49750.0312 - val_loss: 60161.9492
Epoch 4/30
63/63 [==============================] - 1s 17ms/step - loss: 49735.8789 - val_loss: 60148.2891
Epoch 5/30
63/63 [==============================] - 2s 24ms/step - loss: 49722.5117 - val_loss: 60135.1953
Epoch 6/30
63/63 [==============================] - 2s 24ms/step - loss: 49709.5977 - val_loss: 60122.4258
Epoch 7/30
63/63 [==============================] - 1s 17ms/step - loss: 49696.9297 - val_loss: 60109.8672
Epoch 8/30
63/63 [==============================] - 1s 17ms/step - loss: 49684.4453 - val_loss: 60097.4492
Epoch 9/30
63/63 [==============================] - 1s 17ms/step - loss: 49672.0703 - val_loss: 60085.1484
Epoch 10/30
63/63 [=================

In [ ]:
y_pred = model.predict(x_test)

MSE(y_test, y_pred)**(0.5)

1/1 [==============================] - 0s 25ms/step


61014.48584703211